## API Wrapper

In [192]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [4]:
secrets_file = open("C:\\Users\\franc\\OneDrive\\Área de Trabalho\Data Analytics\\Week 2\\6.05 API Wrappers, Spotipy\\secrets.txt","r")
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [5]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [7]:
playlist = sp.user_playlist_tracks("spotify", "1OfeGWEHudCL9mtqoR6Os8")

In [8]:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [9]:
playlist["items"][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [10]:
playlist["total"] 

5550

In [11]:
len(playlist["items"])

100

In [12]:
playlist['next'] #link to the next set of results

'https://api.spotify.com/v1/playlists/1OfeGWEHudCL9mtqoR6Os8/tracks?offset=100&limit=100&additional_types=track'

In [ ]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [13]:
all_tracks = get_playlist_tracks("1OfeGWEHudCL9mtqoR6Os8")
len(all_tracks)

5550

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

Playlist already too big. Too long a processing time for this.

In [118]:
# get track info -  artist, song name and uri:
def get_track_info(tracks):
    
    song_uri = []
    artists = []
    song_name = []
    
    for i in range(len(all_tracks)):
        #get rid of duplicate songs
        if tracks[i]["track"]["uri"] not in song_uri:
            song_uri.append(tracks[i]["track"]["uri"])
            artists.append(tracks[i]["track"]["artists"][0]["name"])
            song_name.append(tracks[i]["track"]["name"])
    
    track_info = {'artist' : artists , 'song' : song_name, 'song_uri':song_uri}
    
    return track_info

In [106]:
def get_features(song_uri):
    features = []
    for i in song_uri:
        features.append(sp.audio_features(i))
    return features

In [119]:
track_info = get_track_info(all_tracks)

In [108]:
len(track_info["song"])

5519

In [109]:
song_uri = track_info['song_uri']

In [173]:
features = get_features(song_uri)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))

In [167]:
len(features)

5519

In [168]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [174]:
features = flatten(features)

TypeError: 'NoneType' object is not iterable

In [176]:
df_track_info = pd.DataFrame(track_info)

In [177]:
df_track_info

,artist,song,song_uri
0,Fiona Apple,Fetch The Bolt Cutters,spotify:track:4SdubskbLiOkpsIjRVVsya
1,Phoebe Bridgers,I Know The End,spotify:track:3cr3oAP4bQFNjZBV7ElKaB
2,Brian Johannesen,Holster Your Silver,spotify:track:31fwgj2HIy7CnQCBfD2wZ5
3,Waxahatchee,Fire,spotify:track:2QUEBOSGpKpL6bzzbQiwwo
4,The Lone Bellow,Count On Me,spotify:track:5sA7NCo4bHH9stpolSymvz
...,...,...,...
5514,The Isley Brothers,Twist And Shout,spotify:track:0URp9jOJtiWKR1AfMGH2Qj
5515,The Isley Brothers,Twist And Shout,spotify:track:0BK8fLeNCMr3TpoWR5dx9V
5516,Grand Funk Railroad,Some Kind Of Wonderful - Remastered 1999,spotify:track:1cU34sZG9kF4FYHCoAD0Ib
5517,Queens of the Stone Age,No One Knows,spotify:track:6NvRxjfYkkT2SpirAlmsjH


In [179]:
df_track_info['features'] = features

In [180]:
df_features = pd.DataFrame(dict(df_track_info['features'])).T

In [181]:
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.485,0.488,0,-11.34,1,0.171,0.526,0.00002,0.0686,0.339,150.856,audio_features,4SdubskbLiOkpsIjRVVsya,spotify:track:4SdubskbLiOkpsIjRVVsya,https://api.spotify.com/v1/tracks/4SdubskbLiOk...,https://api.spotify.com/v1/audio-analysis/4Sdu...,298457,4
1,0.328,0.32,1,-10.282,1,0.035,0.324,0.019,0.234,0.259,111.453,audio_features,3cr3oAP4bQFNjZBV7ElKaB,spotify:track:3cr3oAP4bQFNjZBV7ElKaB,https://api.spotify.com/v1/tracks/3cr3oAP4bQFN...,https://api.spotify.com/v1/audio-analysis/3cr3...,344693,4
2,0.246,0.334,7,-14.365,1,0.0307,0.0154,0.0116,0.147,0.234,81.33,audio_features,31fwgj2HIy7CnQCBfD2wZ5,spotify:track:31fwgj2HIy7CnQCBfD2wZ5,https://api.spotify.com/v1/tracks/31fwgj2HIy7C...,https://api.spotify.com/v1/audio-analysis/31fw...,269500,4
3,0.613,0.454,7,-11.808,1,0.0484,0.681,0.000007,0.137,0.196,85.075,audio_features,2QUEBOSGpKpL6bzzbQiwwo,spotify:track:2QUEBOSGpKpL6bzzbQiwwo,https://api.spotify.com/v1/tracks/2QUEBOSGpKpL...,https://api.spotify.com/v1/audio-analysis/2QUE...,217373,4
4,0.374,0.798,2,-4.66,1,0.111,0.022,0.000006,0.403,0.311,152.058,audio_features,5sA7NCo4bHH9stpolSymvz,spotify:track:5sA7NCo4bHH9stpolSymvz,https://api.spotify.com/v1/tracks/5sA7NCo4bHH9...,https://api.spotify.com/v1/audio-analysis/5sA7...,160600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5514,0.774,0.811,5,-6.526,1,0.0355,0.65,0,0.0736,0.963,123.351,audio_features,0URp9jOJtiWKR1AfMGH2Qj,spotify:track:0URp9jOJtiWKR1AfMGH2Qj,https://api.spotify.com/v1/tracks/0URp9jOJtiWK...,https://api.spotify.com/v1/audio-analysis/0URp...,155400,4
5515,0.689,0.884,5,-6.02,1,0.0356,0.676,0,0.088,0.965,124.252,audio_features,0BK8fLeNCMr3TpoWR5dx9V,spotify:track:0BK8fLeNCMr3TpoWR5dx9V,https://api.spotify.com/v1/tracks/0BK8fLeNCMr3...,https://api.spotify.com/v1/audio-analysis/0BK8...,152173,4
5516,0.855,0.691,2,-6.51,1,0.0514,0.144,0,0.36,0.647,121.472,audio_features,1cU34sZG9kF4FYHCoAD0Ib,spotify:track:1cU34sZG9kF4FYHCoAD0Ib,https://api.spotify.com/v1/tracks/1cU34sZG9kF4...,https://api.spotify.com/v1/audio-analysis/1cU3...,202733,4
5517,0.507,0.588,0,-4.881,1,0.0641,0.0359,0.000129,0.373,0.637,170.889,audio_features,6NvRxjfYkkT2SpirAlmsjH,spotify:track:6NvRxjfYkkT2SpirAlmsjH,https://api.spotify.com/v1/tracks/6NvRxjfYkkT2...,https://api.spotify.com/v1/audio-analysis/6NvR...,278707,4


In [184]:
df_song_features = pd.concat([df_track_info, df_features], axis = 1 )
df_song_features = df_song_features.drop(['features', 'song_uri'], axis = 1)

In [185]:
df_song_features

,artist,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Fiona Apple,Fetch The Bolt Cutters,0.485,0.488,0,-11.34,1,0.171,0.526,0.00002,0.0686,0.339,150.856,audio_features,4SdubskbLiOkpsIjRVVsya,spotify:track:4SdubskbLiOkpsIjRVVsya,https://api.spotify.com/v1/tracks/4SdubskbLiOk...,https://api.spotify.com/v1/audio-analysis/4Sdu...,298457,4
1,Phoebe Bridgers,I Know The End,0.328,0.32,1,-10.282,1,0.035,0.324,0.019,0.234,0.259,111.453,audio_features,3cr3oAP4bQFNjZBV7ElKaB,spotify:track:3cr3oAP4bQFNjZBV7ElKaB,https://api.spotify.com/v1/tracks/3cr3oAP4bQFN...,https://api.spotify.com/v1/audio-analysis/3cr3...,344693,4
2,Brian Johannesen,Holster Your Silver,0.246,0.334,7,-14.365,1,0.0307,0.0154,0.0116,0.147,0.234,81.33,audio_features,31fwgj2HIy7CnQCBfD2wZ5,spotify:track:31fwgj2HIy7CnQCBfD2wZ5,https://api.spotify.com/v1/tracks/31fwgj2HIy7C...,https://api.spotify.com/v1/audio-analysis/31fw...,269500,4
3,Waxahatchee,Fire,0.613,0.454,7,-11.808,1,0.0484,0.681,0.000007,0.137,0.196,85.075,audio_features,2QUEBOSGpKpL6bzzbQiwwo,spotify:track:2QUEBOSGpKpL6bzzbQiwwo,https://api.spotify.com/v1/tracks/2QUEBOSGpKpL...,https://api.spotify.com/v1/audio-analysis/2QUE...,217373,4
4,The Lone Bellow,Count On Me,0.374,0.798,2,-4.66,1,0.111,0.022,0.000006,0.403,0.311,152.058,audio_features,5sA7NCo4bHH9stpolSymvz,spotify:track:5sA7NCo4bHH9stpolSymvz,https://api.spotify.com/v1/tracks/5sA7NCo4bHH9...,https://api.spotify.com/v1/audio-analysis/5sA7...,160600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5514,The Isley Brothers,Twist And Shout,0.774,0.811,5,-6.526,1,0.0355,0.65,0,0.0736,0.963,123.351,audio_features,0URp9jOJtiWKR1AfMGH2Qj,spotify:track:0URp9jOJtiWKR1AfMGH2Qj,https://api.spotify.com/v1/tracks/0URp9jOJtiWK...,https://api.spotify.com/v1/audio-analysis/0URp...,155400,4
5515,The Isley Brothers,Twist And Shout,0.689,0.884,5,-6.02,1,0.0356,0.676,0,0.088,0.965,124.252,audio_features,0BK8fLeNCMr3TpoWR5dx9V,spotify:track:0BK8fLeNCMr3TpoWR5dx9V,https://api.spotify.com/v1/tracks/0BK8fLeNCMr3...,https://api.spotify.com/v1/audio-analysis/0BK8...,152173,4
5516,Grand Funk Railroad,Some Kind Of Wonderful - Remastered 1999,0.855,0.691,2,-6.51,1,0.0514,0.144,0,0.36,0.647,121.472,audio_features,1cU34sZG9kF4FYHCoAD0Ib,spotify:track:1cU34sZG9kF4FYHCoAD0Ib,https://api.spotify.com/v1/tracks/1cU34sZG9kF4...,https://api.spotify.com/v1/audio-analysis/1cU3...,202733,4
5517,Queens of the Stone Age,No One Knows,0.507,0.588,0,-4.881,1,0.0641,0.0359,0.000129,0.373,0.637,170.889,audio_features,6NvRxjfYkkT2SpirAlmsjH,spotify:track:6NvRxjfYkkT2SpirAlmsjH,https://api.spotify.com/v1/tracks/6NvRxjfYkkT2...,https://api.spotify.com/v1/audio-analysis/6NvR...,278707,4


In [195]:
df_song_features.to_csv("df_song_features.csv")